# Set Up
If you haven't setup your credentials in the config.json file, please do so in the given prompt.

In [312]:
import pandas as pd
import requests
import json

In [313]:
# Setting up the config for usage.
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    user_agent = config['user_agent']
    key1 = config['key1']
    key2 = config['key2']
    key3 = config['key3']
    username = config['username']
    print("Config Loaded Successfully.")
    key_list = [key1, key2, key3]
    
except:
    print('config.json wasn\'t found.\nCreating new config file. Please enter your credentials.')
    user_agent = input('Enter User Agent: ')
    username = input('Enter your username: ')
    key1 = input('Enter Secret Key: ')
    key2 = input('Enter Secret Key 2:\nLeave blank if you don\'t have mirror keys. ')
    if key2 == '':
        key2 = key1
    key3 = input('Enter Secret Key 3:\nLeave blank if you don\'t have mirror keys. ')
    if key3 == '':
        key3 = key1
    
#     with open('config.json.sample','r') as f:
#         config = json.load(f)
    config = {'user_agent':None,'key1':None, 'key2':None, 'key3':None, 'username':None}
    config['user_agent'] = user_agent
    config['key1'] = key1
    config['key2'] = key2
    config['key3'] = key3
    config['username'] = username
    
    with open('config.json','w+') as f:
        json.dump(config, f)
    
    key_list = [key1, key2, key3]

    print()
    print("Your configuration has been saved to config.json!")
    
# Fetches variables: user_agent, username, key1, key2, key3, key_list

Config Loaded Successfully.


# Fetching Data from Last.fm

In [314]:
# Generic Function for sending requests

def send_request(user=username, page=1, limit=200, key=key1, method='user.getrecenttracks'):
    
    headers = {'user-agent': user_agent}
    
    payload ={
    'limit':limit,
    'user':user,
    'page': page,
    'api_key': key,
    'method': method,
    'format': 'json'
    }
    
    api_reply = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    
    return api_reply

In [315]:
# Fetch a page and clean data before return.
# Also handles exceptions.

def get_page(num=1, username=username, key=key1, method='user.getrecenttracks'):
    
    try:
        response = send_request(page=num, key=key, method=method)
        
        if response.status_code != 200:
            raise Exception(api_reply.text)
        
        page = (response.json())['recenttracks']['track']
        
        if num == 1: 
            global totalPages
            totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
        
        if (num!=1) & ('@attr' in page[0].keys()): 
            if (page[0]['@attr']=={'nowplaying': 'true'}):
                del page[0]
        
        return page

    except:
        raise Exception("request status {}".format(response.status_code))

# Get the total number of pages for 200 scrobbles per page.

def get_totalPages(method='user.getrecenttracks'):
    response = send_request(method = method)
    totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
    return totalPages

totalPages = get_totalPages()

In [316]:
# Picks a key from given list of keys
def key_picker(num, ls):
    index_num = num % len(ls)
    return ls[index_num]

In [354]:
# Fetches all the pages and merges them into a list "pages"
# Uses a fancy progress bar to show progress.
# Large accounts may time time to fetch!


from IPython.core.display import clear_output

pages = []

for n in range(1, (totalPages)+1):
    
    print('fetched page {}/{}'.format(n, totalPages))
    
    spaces = (((totalPages-n)/totalPages)*50)
    bar = ((n/totalPages)*50)
    string = '[{}{}] {}% Completed'.format('~'*int(bar),' '*int(spaces), int(bar)*2)
    print(string)
    
    clear_output(wait=True)
    
    ky = key_picker(num = n, ls = key_list)
    fetched_page = get_page(num=n, key=ky)
    
    pages = [*pages, *fetched_page]
print()
print("Fetched {} pages, {} scrobbles".format(totalPages, len(pages)))


Fetched 68 pages, 13411 scrobbles


# Cleaning up Fetched Data with #Pandas

In [355]:
df = pd.DataFrame(pages)
df.head()

,artist,streamable,image,mbid,album,name,@attr,url,date
0,"{'mbid': '', '#text': 'Bruno Martini'}",0,"[{'size': 'small', '#text': 'https://lastfm.fr...",b73ce155-5e32-47b8-b5f5-eaba83d83d6b,"{'mbid': '', '#text': 'Wake Up With You'}",Wake Up With You,{'nowplaying': 'true'},https://www.last.fm/music/Bruno+Martini/_/Wake...,NaN
1,{'mbid': '10539fc9-7b6a-4a95-8b9b-cd8d6c7e8a87...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Face Up To The Sun'}",Face Up To The Sun,NaN,https://www.last.fm/music/Mike+Williams/_/Face...,"{'uts': '1645356251', '#text': '20 Feb 2022, 1..."
2,{'mbid': '3e1f2ee4-16be-4406-bf18-6173840cf2b1...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,{'mbid': '4975c513-5b57-455d-94e2-f82aa4922e50...,Spotless,NaN,https://www.last.fm/music/Martin+Garrix/_/Spot...,"{'uts': '1645356052', '#text': '20 Feb 2022, 1..."
3,{'mbid': 'cc91bb2e-7330-4f9e-ae04-4f78f49e3155...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Walk Thru Fire'}",Walk Thru Fire,NaN,https://www.last.fm/music/Vicetone/_/Walk+Thru...,"{'uts': '1645355857', '#text': '20 Feb 2022, 1..."
4,{'mbid': 'c07f0676-9143-4217-8a9f-4c26bd636f13...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",1c6f9d2a-132b-3837-b9c9-52d6489d7000,{'mbid': '44a48c19-f9b7-4fc7-a229-00fd344d4d18...,The Light Behind Your Eyes,NaN,https://www.last.fm/music/My+Chemical+Romance/...,"{'uts': '1645355545', '#text': '20 Feb 2022, 1..."


In [356]:
df.drop(['image','streamable'], inplace=True, axis=1)
if '@attr' in df.columns:
  df.drop('@attr', inplace=True, axis = 1)
df.head()

,artist,mbid,album,name,url,date
0,"{'mbid': '', '#text': 'Bruno Martini'}",b73ce155-5e32-47b8-b5f5-eaba83d83d6b,"{'mbid': '', '#text': 'Wake Up With You'}",Wake Up With You,https://www.last.fm/music/Bruno+Martini/_/Wake...,NaN
1,{'mbid': '10539fc9-7b6a-4a95-8b9b-cd8d6c7e8a87...,,"{'mbid': '', '#text': 'Face Up To The Sun'}",Face Up To The Sun,https://www.last.fm/music/Mike+Williams/_/Face...,"{'uts': '1645356251', '#text': '20 Feb 2022, 1..."
2,{'mbid': '3e1f2ee4-16be-4406-bf18-6173840cf2b1...,,{'mbid': '4975c513-5b57-455d-94e2-f82aa4922e50...,Spotless,https://www.last.fm/music/Martin+Garrix/_/Spot...,"{'uts': '1645356052', '#text': '20 Feb 2022, 1..."
3,{'mbid': 'cc91bb2e-7330-4f9e-ae04-4f78f49e3155...,,"{'mbid': '', '#text': 'Walk Thru Fire'}",Walk Thru Fire,https://www.last.fm/music/Vicetone/_/Walk+Thru...,"{'uts': '1645355857', '#text': '20 Feb 2022, 1..."
4,{'mbid': 'c07f0676-9143-4217-8a9f-4c26bd636f13...,1c6f9d2a-132b-3837-b9c9-52d6489d7000,{'mbid': '44a48c19-f9b7-4fc7-a229-00fd344d4d18...,The Light Behind Your Eyes,https://www.last.fm/music/My+Chemical+Romance/...,"{'uts': '1645355545', '#text': '20 Feb 2022, 1..."


In [357]:
#Split the artist column, and rename the df. Drop the orginal column
artist_split = df['artist'].apply(pd.Series).rename(columns = {'mbid':'artist_mbid','#text':'artist'})
df.drop(columns = ['artist'], inplace=True)

#Split the album column, and rename the df. Drop the original column
album_split = df['album'].apply(pd.Series).rename(columns = {'mbid':'album_mbid','#text':'album'})
df.drop(columns = ['album'], inplace=True)

#Split the date column, and rename the df. Drop the original column
#Also check if residual '0' column is made. (If the top song is being streamed, it wont include the timing.)
date_split = df['date'].apply(pd.Series).rename(columns = {'uts':'uts','#text':'date'})
df.drop(columns = ['date'], inplace=True)
if 0 in date_split.columns:
    date_split.drop(0, axis=1, inplace=True)

#Merging the new columns into main df
df = pd.concat([df, artist_split, album_split, date_split],axis=1)
df.head()

,mbid,name,url,artist_mbid,artist,album_mbid,album,date,uts
0,b73ce155-5e32-47b8-b5f5-eaba83d83d6b,Wake Up With You,https://www.last.fm/music/Bruno+Martini/_/Wake...,,Bruno Martini,,Wake Up With You,NaN,NaN
1,,Face Up To The Sun,https://www.last.fm/music/Mike+Williams/_/Face...,10539fc9-7b6a-4a95-8b9b-cd8d6c7e8a87,Mike Williams,,Face Up To The Sun,"20 Feb 2022, 11:24",1645356251
2,,Spotless,https://www.last.fm/music/Martin+Garrix/_/Spot...,3e1f2ee4-16be-4406-bf18-6173840cf2b1,Martin Garrix,4975c513-5b57-455d-94e2-f82aa4922e50,Seven,"20 Feb 2022, 11:20",1645356052
3,,Walk Thru Fire,https://www.last.fm/music/Vicetone/_/Walk+Thru...,cc91bb2e-7330-4f9e-ae04-4f78f49e3155,Vicetone,,Walk Thru Fire,"20 Feb 2022, 11:17",1645355857
4,1c6f9d2a-132b-3837-b9c9-52d6489d7000,The Light Behind Your Eyes,https://www.last.fm/music/My+Chemical+Romance/...,c07f0676-9143-4217-8a9f-4c26bd636f13,My Chemical Romance,44a48c19-f9b7-4fc7-a229-00fd344d4d18,Number Three,"20 Feb 2022, 11:12",1645355545


In [358]:
#Reordering columns
col_reorder = ['artist','name','album','date','artist_mbid','mbid','album_mbid','uts','url']
df = df.reindex(col_reorder, axis=1)
df.head()

,artist,name,album,date,artist_mbid,mbid,album_mbid,uts,url
0,Bruno Martini,Wake Up With You,Wake Up With You,NaN,,b73ce155-5e32-47b8-b5f5-eaba83d83d6b,,NaN,https://www.last.fm/music/Bruno+Martini/_/Wake...
1,Mike Williams,Face Up To The Sun,Face Up To The Sun,"20 Feb 2022, 11:24",10539fc9-7b6a-4a95-8b9b-cd8d6c7e8a87,,,1645356251,https://www.last.fm/music/Mike+Williams/_/Face...
2,Martin Garrix,Spotless,Seven,"20 Feb 2022, 11:20",3e1f2ee4-16be-4406-bf18-6173840cf2b1,,4975c513-5b57-455d-94e2-f82aa4922e50,1645356052,https://www.last.fm/music/Martin+Garrix/_/Spot...
3,Vicetone,Walk Thru Fire,Walk Thru Fire,"20 Feb 2022, 11:17",cc91bb2e-7330-4f9e-ae04-4f78f49e3155,,,1645355857,https://www.last.fm/music/Vicetone/_/Walk+Thru...
4,My Chemical Romance,The Light Behind Your Eyes,Number Three,"20 Feb 2022, 11:12",c07f0676-9143-4217-8a9f-4c26bd636f13,1c6f9d2a-132b-3837-b9c9-52d6489d7000,44a48c19-f9b7-4fc7-a229-00fd344d4d18,1645355545,https://www.last.fm/music/My+Chemical+Romance/...


# Exporting into CSV

In [360]:
import copy

# df.to_csv('export.csv', encoding='utf-8')
# df.to_json('export.json', encoding='utf-8')

df_ = copy.deepcopy(df)

# Data Analysis & Visualization

In [363]:
# Finding Top Songs:
TopSongs = pd.DataFrame(df['name'].value_counts())
TopSongs.head(10)

,name
Fu re te Fu re ru,52
us,50
Signal,50
contrast,41
打上花火,40
だから僕は音楽を辞めた,40
Shandy,39
ninelie,39
katharsis,38
Who What Who What,37


In [364]:
TopArtists = pd.DataFrame(df['artist'].value_counts())
TopArtists.head(10)

,artist
Yutaka Yamada,822
TK from Ling tosite sigure,530
Ling Tosite Sigure,426
ヨルシカ,308
Aimer,306
TK from 凛として時雨,296
Hiroyuki Sawano,291
小田朋美,242
amazarashi,223
RADWIMPS,179


# Working with Timeseries Data

In [370]:
# 20 Feb 2022, 10:03
import datetime as dt
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'] + dt.timedelta(hours=5, minutes=30)

In [371]:
map1 = (df['date'].dt.time > dt.time(00,00,00))
map2 = (df['date'].dt.time < dt.time(23,59,00))
map3 = (df['date'].dt.date == dt.date(2022,2,20))
df[map3]
# df_time

,artist,name,album,date,artist_mbid,mbid,album_mbid,uts,url
1,Mike Williams,Face Up To The Sun,Face Up To The Sun,2022-02-20 22:24:00,10539fc9-7b6a-4a95-8b9b-cd8d6c7e8a87,,,1645356251,https://www.last.fm/music/Mike+Williams/_/Face...
2,Martin Garrix,Spotless,Seven,2022-02-20 22:20:00,3e1f2ee4-16be-4406-bf18-6173840cf2b1,,4975c513-5b57-455d-94e2-f82aa4922e50,1645356052,https://www.last.fm/music/Martin+Garrix/_/Spot...
3,Vicetone,Walk Thru Fire,Walk Thru Fire,2022-02-20 22:17:00,cc91bb2e-7330-4f9e-ae04-4f78f49e3155,,,1645355857,https://www.last.fm/music/Vicetone/_/Walk+Thru...
4,My Chemical Romance,The Light Behind Your Eyes,Number Three,2022-02-20 22:12:00,c07f0676-9143-4217-8a9f-4c26bd636f13,1c6f9d2a-132b-3837-b9c9-52d6489d7000,44a48c19-f9b7-4fc7-a229-00fd344d4d18,1645355545,https://www.last.fm/music/My+Chemical+Romance/...
5,Virtual Riot,REDLINE - color bass doggo VIP,Simulation (Deluxe Version),2022-02-20 22:09:00,,,,1645355370,https://www.last.fm/music/Virtual+Riot/_/REDLI...
...,...,...,...,...,...,...,...,...,...
83,TK from Ling tosite sigure,Dramatic Slow Motion (Reconstructed 2020),Dramatic Slow Motion (Reconstructed 2020),2022-02-20 01:19:00,,,,1645280376,https://www.last.fm/music/TK+from+Ling+tosite+...
84,TK from Ling tosite sigure,Dramatic Slow Motion (Reconstructed 2020),Dramatic Slow Motion (Reconstructed 2020),2022-02-20 01:15:00,,,,1645280148,https://www.last.fm/music/TK+from+Ling+tosite+...
85,YOASOBI,RGB,RGB,2022-02-20 01:10:00,df6c619f-4334-43e2-8b6a-4a32af1e4f85,7d9612d7-39b6-47d8-a1d7-e3cb63eac257,40391a43-8963-4b72-a848-9b1135e027f8,1645279850,https://www.last.fm/music/YOASOBI/_/RGB
86,Aimer,Brave Shine,DAWN,2022-02-20 01:10:00,9388cee2-7d57-4598-905f-106019b267d3,115293ba-0f60-4fc9-97ab-3a613ff85c34,5e0e3da4-41d7-4cc1-9454-f8f321bbc344,1645279849,https://www.last.fm/music/Aimer/_/Brave+Shine
